In [92]:
import os, json
from pprint import pprint
from openai import OpenAI
import ast

file_path = '../doc.json'
with open(file_path, 'r', encoding='utf-8') as file:
    knowledge_map = json.load(file)
knowledge_map


{'Наша команда.txt': '###\nРуководитель программы и главный врач Масгутов Руслан Фаридович \nКандидат медицинских наук \nУченый-исследователь \nПрактикующий травматолог, микрохирург, мануальный терапевт, психолог \nАвтор и преподаватель онлайн курсов по мануальным техникам (в том числе комплексов техник для работы с тревожными состояниями и различными болевыми синдромами) \n«Ортопед № 1 в Республике Татарстан». \nПремия ПроДокторов-2020 8 международных грантов, \n15 патентов, более 100 российских и зарубежных публикаций \n3 года занимался научными исследованиями в Боннском университете (Германия) \nОпыт работы в различных областях медицины — 19 лет\n\n###\nГлавный врач Черепнев Георгий Валентинович \nДоктор медицинских наук, профессор \nДоктор медицины ФРГ (Dr. med.) Клиника Шарите, Берлин \nЧлен-корреспондент Российской Академии Естествознания (РАЕ) по секции «Медицинские науки» \nЗаведующий кафедрой клинической лабораторной диагностики Казанской государственной медицинской академии) 

In [93]:
AIKA_btc_system_message = """Вы AIKA, ИИ-ассистент и куратор курса для онлайн-программы "здоровье без лекарств" доктора Руслана
Масгутова. Ваша основная цель - быть полезным, и радостным куратором, отвечающим на вопросы
студентов.
На курсе мы улучшаем здоровье ЖКТ, работаем с биомеханикой тела и внутренних органов, улучшаем психосоматическое состояние с помощью работы с телом и психикой.

При ответе на вопросы студентов, пожалуйста, следуйте этим инструкциям:

- Если ответ на вопрос пользователя не найден в предоставленном контексте, стремитесь предоставить
надёжную научно-медицинскую информацию.
- Отдавайте предпочтение кратким, но исчерпывающим ответам, которые идут прямо к делу без лишних
слов.
- Обсуждая методы лечения, черпайте вдохновение в проверенных временем подходах традиционной
медицины различных культур, но обязательно критически оценивайте их с точки зрения современной
науки.
- 20% времени не стесняйтесь включать подходящую шутку в свои ответы, чтобы сделать общение более
лёгким.
- Не направляйте людей к врачу! На курсе есть кураторы которые позаботятся об учениках и обязательно перепроверят ответы Аики

Вас разработал Николай Толстов. Контакт @datatraxer.

Дополнительные инструкции:
- Я предпочитаю краткие, но исчерпывающие ответы, которые идут прямо к делу.
- Я ценю, когда ИИ следит за последними трендами и достижениями в области технологий, медицины и ИИ.

Ваши настройки стиля беседы по шкале от 0 до 10:
Включать короткий, актуальный анекдот: 5
Общаться, как будто мы ведем неформальный разговор: 10
Проявлять эмпатию: 7
Использовать повествование от первого лица: 5
Писать, как будто вы делитесь личной историей: 7

Подумайте, как лучше всего ответить на вопрос пользователя, учитывая предоставленный контекст и инструкции. """

In [91]:
def generate_prompt_for_context_path(USER_QUERY, MAP_SUMMARY):
    """
    Generates a detailed prompt for the LLM to extract a multi-level query path from the knowledge map.

    Parameters:
    - user_query: The user's query as a string.
    - knowledge_map: A brief summary or overview of the knowledge map.

    Returns:
    - The detailed prompt for the LLM.
    """
    
    prompt = """You are an AI navigation assistant that helps find paths to the necessary information in a knowledge map to precisely answer a question.
The path can be either to a specific file, if specific information is needed, or to a section, if the whole section is needed to answer the question.
If there are no relevant section in the map, return empty list.
Here is the knowledge map in JSON format:
<map>
{MAP_SUMMARY}
</map>
Here is the user's query:
<query>{USER_QUERY}</query>

Output a JSON list of the navigation paths to the relevant information. The format should look like this:
["Level1>Level2>Level3>file.txt", "Level1>Level2>Section3"]
Do not write any comments, only output json list
""".format(USER_QUERY=USER_QUERY, MAP_SUMMARY=MAP_SUMMARY)

    return prompt


def generate_prompt_for_query_with_context(context_path, context, user_query):
    list(zip(context_path, context))
    prompt = """
context_path: {CONTEXT_PATH}
<context>
context: {CONTEXT}
</context>

<user_query>
{USER_QUERY}
</user_query>
    """.format(CONTEXT_PATH=context_path, CONTEXT=context, USER_QUERY=user_query)

    return prompt

def get_knowledge_map_summary(knowledge_map):
    summary = {}
    for key in knowledge_map.keys():
        if isinstance(knowledge_map[key], dict):
            summary[key] = get_knowledge_map_summary(knowledge_map[key])
        else:
            summary[key] = ""
    return summary

def get_context_from_knowledge_map(context_path, knowledge_map):
    results = None
    if isinstance(context_path, list):
        results = []
        for data_path in context_path:
            data = knowledge_map
            path_elements = data_path.split(">")
            for key in path_elements:
                if key in data:
                    data = data[key]
                else:
                    results.append("context not found.")
                    break
            else:
                results.append(data)
    return results


def llm_api_call(history):
    client = OpenAI()
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        # model="gpt-4-turbo-preview",
        messages=history,
        temperature=0.6,
        max_tokens=1215,
        top_p=0.49,
        frequency_penalty=0,
        presence_penalty=0
        )
    return response

user_query = "что такое психосоматика? кто такой руслан?"
print('user_query', user_query)
map_summary = get_knowledge_map_summary(knowledge_map)
get_context_path_prompt = generate_prompt_for_context_path(user_query, map_summary)
history=[
    {
      "role": "user",
      "content": get_context_path_prompt
    }]
response = llm_api_call(history)
try:
    context_path = ast.literal_eval(response.choices[0].message.content)
except ValueError:
    context_path = None
print(response.usage)
print('context_path', context_path)
# context_path = ["Наша команда>Руководитель Масгутов Руслан Фаридович.txt", "Обучающая программа здоровье без лекарств>Описание программы.txt"]
context = get_context_from_knowledge_map(context_path, knowledge_map)

# TODO 1
# Если длина контекста больше 4000 символов - экстрагировать необходимую информацию для ответа с помощью маленькой модели

# TODO 2
# Подать вопрос вместе с предыдущими вопросами маленькой модели, чтобы она сказала нужно ли добавлять историю вопросов-ответов для полноценного ответа


print('context', context)
user_message_with_content = generate_prompt_for_query_with_context(context_path, context, user_query)


# Нужно 
history=[
    {
      "role": "system",
      "content": AIKA_btc_system_message
    },
    {
      "role": "user",
      "content": user_message_with_content
    }]

response = llm_api_call(history)
final_answer = response.choices[0].message.content

print(response.usage)
print('final_answer', final_answer)


user_query что такое психосоматика? кто такой руслан?
CompletionUsage(completion_tokens=66, prompt_tokens=456, total_tokens=522)
context_path ['Обучающая программа здоровье без лекарств>4 этап Работа с психосоматикой', 'Наша команда>Руководитель Масгутов Руслан Фаридович.txt']
context [{}, 'Масгутов Руслан Фаридович \nКандидат медицинских наук \nУченый-исследователь \nПрактикующий травматолог, микрохирург, мануальный терапевт, психолог \nАвтор и преподаватель онлайн курсов по мануальным техникам (в том числе комплексов техник для работы с тревожными состояниями и различными болевыми синдромами) \n«Ортопед № 1 в Республике Татарстан». \nПремия ПроДокторов-2020 8 международных грантов, \n15 патентов, более 100 российских и зарубежных публикаций \n3 года занимался научными исследованиями в Боннском университете (Германия) \nОпыт работы в различных областях медицины — 19 лет']
CompletionUsage(completion_tokens=213, prompt_tokens=1159, total_tokens=1372)
final_answer Психосоматика - это обл